In [6]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from pathlib import Path

# Argument parser

print('[INFO] Starting System...')
print('[INFO] Importing pretrained model...')
pose_predictor_68_point = dlib.shape_predictor("pretrained_model/shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("pretrained_model/dlib_face_recognition_resnet_model_v1.dat")
face_detector = dlib.get_frontal_face_detector()
print('[INFO] Pretrained model imported.')

def transform(image, face_locations):
    coord_faces = []
    for face in face_locations:
        rect = face.top(), face.right(), face.bottom(), face.left()
        coord_face = max(rect[0], 0), min(rect[1], image.shape[1]), min(rect[2], image.shape[0]), max(rect[3], 0)
        coord_faces.append(coord_face)
    return coord_faces

def encode_face(image):
    if len(image.shape) != 3:
        print(f"[ERROR] The input image does not have three dimensions. Current shape: {image.shape}")
        return {}

    # Ensure the image is in RGB format
    if image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    elif image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    else:
        print(f"[ERROR] Unsupported number of channels: {image.shape[2]}.")
        return {}

    # Ensure the image is in the correct format
    if image.dtype != np.uint8:
        image = image.astype(np.uint8)
    
    try:
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    except Exception as e:
        print(f"[ERROR] Failed to convert image to grayscale. Error: {e}")
        return {}

    try:
        face_locations = face_detector(gray_image, 1)
    except Exception as e:
        print(f"[ERROR] Failed to detect faces. Error: {e}")
        return {}

    if len(face_locations) == 0:
        print("[WARNING] No faces detected in the image.")
        return {}

    
    landmarks_list = []
    for face_location in face_locations:
        shape = pose_predictor_68_point(image, face_location)
        #face_encodings_list.append(np.array(face_encoder.compute_face_descriptor(image, shape, num_jitters=1)))
        shape = face_utils.shape_to_np(shape)
        landmarks_list.append(shape)
    
    face_locations = transform(image, face_locations)
    return  {"facelocations":face_locations, "landmarklist":landmarks_list}

def easy_face_reco(frame, ):
    rgb_small_frame = frame[:, :, ::-1]
    data= encode_face(rgb_small_frame)
    if data:
        face_locations_list=data["facelocations"]
        landmarks_list=data["landmarklist"]
        name = "Unknown"
            
        if len(face_locations_list)>0:
            for (top, right, bottom, left) in face_locations_list:
                color = (0, 255, 0) #if name != "Unknown" else (0, 0, 255)  # Green for known, Red for unknown
                #cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
                #cv2.rectangle(frame, (left, bottom - 30), (right, bottom), color, cv2.FILLED)
                cv2.putText(frame, name, (left + 2, bottom - 2), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2)

        #landmarks visualization
        # for shape in landmarks_list:
        #     for (x, y) in shape:
        #         cv2.circle(frame, (x, y), 1, (255, 0, 255), -1)

if __name__ == '__main__':
    


    

    video_capture = cv2.VideoCapture("../../cv/face_recognition/videos/vid2.mp4")
    print('[INFO] Webcam well started')
    print('[INFO] Detecting...')

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("[ERROR] Failed to capture image from webcam.")
            break

        easy_face_reco(frame)
        cv2.imshow('Easy Facial Recognition App', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print('[INFO] Stopping System')
    video_capture.release()
    cv2.destroyAllWindows()


[INFO] Starting System...
[INFO] Importing pretrained model...
[INFO] Pretrained model imported.
[INFO] Webcam well started
[INFO] Detecting...
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No faces detected in the image.
[WARNING] No fac

In [2]:
import os

import PIL.Image
import cv2
import dlib
import numpy as np
from imutils import face_utils
from pathlib import Path
import PIL
class LandMarkDatasetMaker:
    def __init__(self,dataset_path,landmark_weight_path,fr_weight_path,directory_to_save) :
        self.root=Path(dataset_path)
        all_names=os.listdir(dataset_path)
        self.data=self.set_data_paths(all_names=all_names)
        self.landmark_weight_path=landmark_weight_path
        self.fr_weight_path=fr_weight_path
        self.directory_to_save=Path(directory_to_save)
        self.load_pretrained_weight()


    def load_pretrained_weight(self):
        try:
            self.pose_predictor_68_point = dlib.shape_predictor(self.landmark_weight_path)
            self.face_encoder = dlib.face_recognition_model_v1(self.fr_weight_path)
            self.face_detector = dlib.get_frontal_face_detector()
            print(f"weight loaded successfully")
        except  Exception as e:
            print(f"loading weight exception occur {e}")

    def set_data_paths(self,all_names):
        data_paths={}
        for name in all_names:
            new_path=self.root.joinpath(name)
            images_path=list(new_path.rglob("*.jpg")) + list(new_path.rglob("*.png"))
            data_paths[name]=images_path
        return data_paths
    def get_data(self,):
        return self.data
    def get_gray_img(self,image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        return gray_image

    def get_transform_image(self,image):
        assert len(image.shape) == 3, f"[ERROR] The input image does not have three dimensions. Current shape: {image.shape}"
        
        if image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        elif image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        elif image.shape[2] == 1:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        else:
            assert False,f"[ERROR] Unsupported number of channels: {image.shape[2]}." 
        
        if image.dtype != np.uint8:
            image = image.astype(np.uint8)
        gray_image=self.get_gray_img(image)
        return gray_image
    def make_landmark_list(self,face_locations,image):
        landmarks_list = []
        for face_location in face_locations:
            try: 
                shape = self.pose_predictor_68_point(image, face_location)
                #face_encodings_list.append(np.array(face_encoder.compute_face_descriptor(image, shape, num_jitters=1)))
                shape = face_utils.shape_to_np(shape)
                landmarks_list.append(shape) 
            except Exception as e:
                print(f"exception:{e}")
        return landmarks_list     
    def get_face_landmarks(self,image)->list:
        
    
            

        # Ensure the image is in the correct format

        gray_image=self.get_transform_image(image)
        
        face_locations=list()
        try:
            face_locations =self.face_detector(gray_image, 1)
        except Exception as e:
            print(f"[ERROR] Failed to detect faces. Error: {e}")
            return []

        if len(face_locations) == 0:
            print("[WARNING] No faces detected in the image.")
            return []

        return  self.make_landmark_list(face_locations,image)
    def save_landmarks(self, landmarks, img_path)->None:
        user_directory = self.directory_to_save / Path(img_path).parent.name
        
        if not user_directory.exists():
            user_directory.mkdir(parents=True, exist_ok=True)
        
        # Save landmarks as .npy file
        save_path = user_directory / (Path(img_path).stem + "_landmarks.npy")
        np.save(save_path, landmarks)
        print(f"[INFO] Landmarks saved for {img_path} at {save_path}")
    def save_landmarks_of_all_images(self,)->None:
        
        for user_name in self.data:
            for img_path in self.data[user_name]:
                image = PIL.Image.open(img_path)
                image = np.array(image)
                
                # Check if the image is empty or has an unexpected shape
                if image.size == 0:
                    print(f"[ERROR] The image {img_path} is empty or cannot be read.")
                    continue
                
                landmark_list =self.get_face_landmarks(image)
                if len( landmark_list)==0:
                    print(f"[WARNING] No faces detected in {img_path}. Skipping this file.")
                    continue 
                self.save_landmarks(landmark_list, img_path)


        
         
        
obj=LandMarkDatasetMaker(
    "../face_recognition/dataset_not_for_upload_images",
    "pretrained_model/shape_predictor_68_face_landmarks.dat",
    "pretrained_model/dlib_face_recognition_resnet_model_v1.dat",
    "./dataset_not_for_upload_images"
    )
obj.save_landmarks_of_all_images()

weight loaded successfully
[INFO] Landmarks saved for ..\face_recognition\dataset_not_for_upload_images\Abdullah Munir\cropped_face_0_0.png at dataset_not_for_upload_images\Abdullah Munir\cropped_face_0_0_landmarks.npy
[INFO] Landmarks saved for ..\face_recognition\dataset_not_for_upload_images\Abdullah Munir\cropped_face_100_0.png at dataset_not_for_upload_images\Abdullah Munir\cropped_face_100_0_landmarks.npy
[INFO] Landmarks saved for ..\face_recognition\dataset_not_for_upload_images\Abdullah Munir\cropped_face_101_0.png at dataset_not_for_upload_images\Abdullah Munir\cropped_face_101_0_landmarks.npy
[INFO] Landmarks saved for ..\face_recognition\dataset_not_for_upload_images\Abdullah Munir\cropped_face_102_0.png at dataset_not_for_upload_images\Abdullah Munir\cropped_face_102_0_landmarks.npy
[INFO] Landmarks saved for ..\face_recognition\dataset_not_for_upload_images\Abdullah Munir\cropped_face_103_0.png at dataset_not_for_upload_images\Abdullah Munir\cropped_face_103_0_landmarks.n

TypeError: unsupported operand type(s) for +: 'generator' and 'generator'